<a href="https://colab.research.google.com/github/chrisliii/FINE-452/blob/main/Fama_French.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
stock_name = "^GSPC"
ticker = yf.Ticker(stock_name)
#ticker.info
# monthly data
df_ticker_monthly_return = ticker.history(period="max", interval='1mo')
# monthly return
df_ticker_monthly_return['MonthlyReturn'] = 100*( (df_ticker_monthly_return['Close'].shift(-1)).div(df_ticker_monthly_return['Close']) - 1)
df_ticker_monthly_return['MonthlyReturn'] = df_ticker_monthly_return['MonthlyReturn'].shift(1)
df_ticker_monthly_return = df_ticker_monthly_return.dropna()
#date to string
df_ticker_monthly_return = df_ticker_monthly_return.reset_index()
df_ticker_monthly_return['Date'] = df_ticker_monthly_return['Date'].apply(lambda x: x.strftime('%Y%m'))

df_ticker_monthly_return.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn
0,192801,17.76,17.76,17.26,17.57,0,0,0,-0.509626
1,192802,17.53,17.63,16.95,17.26,0,0,0,-1.764371
2,192803,17.30,19.28,17.30,19.28,0,0,0,11.703360
3,192804,18.91,19.75,18.91,19.75,0,0,0,2.437759
4,192805,19.78,20.44,19.36,20.00,0,0,0,1.265823


In [ ]:
stock_name = "ADBE"
ticker = yf.Ticker(stock_name)
#ticker.info
# monthly data
df_ticker_monthly_return = ticker.history(period="max", interval='1mo')
# process Dividends
df_ticker_monthly_return['Dividends'] = df_ticker_monthly_return['Dividends'].shift(1)
df_ticker_monthly_return = df_ticker_monthly_return.dropna()
# monthly return
df_ticker_monthly_return['MonthlyReturn'] = 100*( (df_ticker_monthly_return['Close'].shift(-1)+df_ticker_monthly_return['Dividends'].shift(-1)).div(df_ticker_monthly_return['Close']) - 1)
df_ticker_monthly_return['MonthlyReturn'] = df_ticker_monthly_return['MonthlyReturn'].shift(1)
df_ticker_monthly_return = df_ticker_monthly_return.dropna()
#date to string
df_ticker_monthly_return = df_ticker_monthly_return.reset_index()
df_ticker_monthly_return['Date'] = df_ticker_monthly_return['Date'].apply(lambda x: x.strftime('%Y%m'))

df_ticker_monthly_return.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn
0,198610,0.0,0.27,0.20,0.27,26118400.0,0.0,0.0,35.000000
1,198611,0.0,0.28,0.25,0.28,18368000.0,0.0,0.0,3.703704
2,198612,0.0,0.31,0.26,0.29,46156800.0,0.0,0.0,3.571429
3,198701,0.0,0.42,0.29,0.39,62105600.0,0.0,0.0,34.482759
4,198702,0.0,0.53,0.39,0.52,71571200.0,0.0,0.0,33.333333


In [5]:
FF_Research_source = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip'
df_FF_Research = pd.read_csv(FF_Research_source, encoding="utf-8", skipinitialspace=True)
df_FF_Research.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']
df_FF_Research = df_FF_Research.dropna()
df_FF_Research['Mkt-RF'] = df_FF_Research['Mkt-RF'].astype(float)
df_FF_Research['SMB'] = df_FF_Research['SMB'].astype(float)
df_FF_Research['HML'] = df_FF_Research['HML'].astype(float)

df_FF_Research.head()

ParserError: ignored

In [ ]:
df_ticker_FF = pd.merge(df_ticker_monthly_return, df_FF_Research, on='Date')
df_ticker_FF['Date'] = df_ticker_FF['Date'].astype(int)
#df_ticker_FF = df_ticker_FF[df_ticker_FF.Date > 201000]

df_ticker_FF.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn,Mkt-RF,SMB,HML,RF
0,198610,0.0,0.27,0.20,0.27,26118400.0,0.0,0.0,35.000000,4.66,-2.48,-1.32,0.46
1,198611,0.0,0.28,0.25,0.28,18368000.0,0.0,0.0,3.703704,1.17,-1.92,-0.06,0.39
2,198612,0.0,0.31,0.26,0.29,46156800.0,0.0,0.0,3.571429,-3.27,0.08,0.37,0.49
3,198701,0.0,0.42,0.29,0.39,62105600.0,0.0,0.0,34.482759,12.47,-1.81,-3.18,0.42
4,198702,0.0,0.53,0.39,0.52,71571200.0,0.0,0.0,33.333333,4.39,3.49,-5.99,0.43


In [ ]:
model = sm.OLS(df_ticker_FF['MonthlyReturn'], sm.add_constant(
    df_ticker_FF[['Mkt-RF', 'SMB', 'HML']].values))
result = model.fit()
print(stock_name + '\n')
print (result.params)
print(result.summary())

ADBE

const    2.020638
x1       1.301376
x2       0.632951
x3      -0.539995
dtype: float64
                            OLS Regression Results                            
Dep. Variable:          MonthlyReturn   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     48.18
Date:                Wed, 30 Sep 2020   Prob (F-statistic):           1.28e-26
Time:                        20:15:21   Log-Likelihood:                -1576.7
No. Observations:                 406   AIC:                             3161.
Df Residuals:                     402   BIC:                             3177.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [ ]:
df_ticker_monthly_return.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn
404,202006,387.08,446.15,382.00,435.31,70625900.0,0.0,0.0,12.599586
405,202007,434.80,470.61,416.29,444.32,54649300.0,0.0,0.0,2.069789
406,202008,449.84,533.70,431.59,513.39,54517800.0,0.0,0.0,15.545103
407,202009,515.00,536.88,452.52,489.33,81272000.0,0.0,0.0,-4.686496
408,202009,488.13,497.15,486.22,490.43,3377305.0,0.0,0.0,0.224797
